In [137]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [138]:
import numpy as np
import pandas as pd
import ast
import warnings
warnings.filterwarnings("ignore")

In [139]:
dataset = pd.read_csv('/content/drive/MyDrive/Deep_learning/airline-price-prediction.csv')
#dataset = pd.read_csv('airline-price-prediction.csv')
dataset.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,5/3/2022,Vistara,UK,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...","56,588"
1,18-03-2022,Vistara,UK,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}","59,649"
2,9/3/2022,GO FIRST,G8,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}","3,916"
3,15-03-2022,GO FIRST,G8,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}","6,031"
4,22-03-2022,Vistara,UK,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}","7,081"


In [140]:
dataset['full_information_timestamp'] = 0
dataset['flight_day'] = 0
dataset['flight_month'] = 0
dataset['week_day_of_flight'] = 0
dataset['num_of_hours_taken'] =0
dataset['num_of_stops'] = 0
dataset['distance_between_2_cities'] = 0
dataset['source'] = 0
dataset['destination'] = 0
dataset['formatted_price'] =0

In [141]:
def format_date(date):
    dashed_date = date.replace('/','-')
    return dashed_date

In [142]:
dataset['full_information_timestamp'] = dataset.date.apply(lambda date:format_date(date))

In [143]:
def extract_day_month(date):
  splitted_date = date.split('-')
  day = int(splitted_date[0])
  month =int(splitted_date[1])
  if splitted_date[1]== '2' or splitted_date[1]== '02':
      new_date_format = 'February ' + splitted_date[0] + ', '+ splitted_date[2]
  else :
      new_date_format = 'March ' + splitted_date[0] + ', '+ splitted_date[2]
  return day,month,new_date_format

In [144]:
dataset[['flight_day', 'flight_month','full_information_timestamp']] = pd.DataFrame(dataset.full_information_timestamp.apply(lambda date: extract_day_month(date)).tolist(), index=dataset.index)

In [145]:
dataset['full_information_timestamp'] = pd.to_datetime(dataset['full_information_timestamp'])

In [148]:
dataset.sort_values(by ='full_information_timestamp',inplace =True,kind = 'mergesort',ascending = True)

In [149]:
def extract_weekday(day):
    return day.day_name()

In [150]:
dataset['week_day_of_flight'] = dataset.full_information_timestamp.apply(lambda date:extract_weekday(date))

In [151]:
dataset['full_information_timestamp'] = pd.to_datetime(dataset['full_information_timestamp'])

In [152]:
def extract_weekday(day):
    return day.day_name()

In [153]:
dataset['week_day_of_flight'] = dataset.full_information_timestamp.apply(lambda date:extract_weekday(date))

In [154]:
def remove_low_frequency_airlines():
    high_frequency_airlines =['Vistara','Air India','Indigo']
    dataset ['airline'] = [airline if airline in high_frequency_airlines else 'Other_airline' for airline in dataset['airline']]
   

In [155]:
remove_low_frequency_airlines()

In [156]:
def calculate_time_taken(time_taken):
      time = time_taken.split(' ')
      float_hour = time[0].split('.')
      hours = 0
      if(len(float_hour)>1):
          time[0] = float_hour[0] + 'h'
          time[1]= float_hour[1][:-1] + time[1]
      if time[1][:-1] != '' : 
          if int(time[1][:-1]) >= 40 :
              hours = int(time[0][:-1]) + 1
          else :
              hours = int(time[0][:-1]) 
      else :
          hours = int(time[0][:-1])
      return hours

In [157]:
dataset['num_of_hours_taken'] = dataset.time_taken.apply(lambda time_taken: calculate_time_taken(time_taken))

In [158]:
def categorize_time(time):
    hours = time.split(':')
    hours[0] = int(hours[0])
    if hours[0] <6:
        return 'Early morning'
    elif 6<= hours[0] < 12 :
        return 'Morning'
    elif 12<= hours[0] < 18:
        return 'Afternoon'
    else:
        return 'Evening'

In [159]:
dataset['dep_time'] = dataset.dep_time.apply(lambda time: categorize_time(time))

In [160]:
dataset['arr_time'] = dataset.arr_time.apply(lambda time: categorize_time(time))

In [161]:
def split_num_of_stops(stop):
    if stop[:8] == 'non-stop':
        stop_count = 0
    elif stop[:6] == '1-stop':
        stop_count = 1
    else:
        stop_count = 2
    return stop_count

In [162]:
dataset['num_of_stops']= dataset.stop.apply(lambda stop:split_num_of_stops(stop))

In [163]:
dataset['type'] = [1 if kind_of_trip == 'business' else 0 for kind_of_trip in dataset['type']]

In [164]:
def split_route(route):
    path = ast.literal_eval(route)
    source = path['source']
    destination = path['destination']
    return source,destination

In [165]:
dataset[['source', 'destination']] = pd.DataFrame(dataset.route.apply(lambda route: split_route(route)).tolist(), index=dataset.index)

In [166]:
distances = dict()
distances['Mumbai','Kolkata'] = 2167
distances['Mumbai','Hyderabad'] = 721
distances['Mumbai','Chennai'] = 1344
distances['Mumbai','Bangalore'] = 995
distances['Mumbai','Delhi'] = 1439
distances['Delhi','Bangalore'] = 2169
distances['Delhi','Kolkata'] = 1554
distances['Delhi','Hyderabad'] = 1579
distances['Delhi','Chennai'] = 2202
distances['Bangalore','Kolkata']= 1560
distances['Bangalore','Hyderabad'] = 569
distances['Bangalore','Chennai'] = 348
distances['Kolkata','Hyderabad'] = 1489
distances['Kolkata','Chennai'] = 1663
distances['Hyderabad','Chennai'] = 628

In [167]:
def calculate_distance(source,destination):
    try :
        dist = distances[source,destination]
    except :
        dist = distances[destination,source]
    return dist

In [168]:
dataset['distance_between_2_cities'] = dataset.apply(lambda dataFrame: calculate_distance(dataFrame['source'], dataFrame['destination']), axis=1)
dataset['distance_between_2_cities'] = dataset['distance_between_2_cities']/2202

In [169]:
def fix_price_format(price):
    number = price.split(',')
    cost = number[0] + number[1]
    cost=int(cost)
    return cost

In [170]:
dataset['formatted_price'] = dataset.price.apply(lambda price:fix_price_format(price))

In [171]:
dataset.sort_values(by ='full_information_timestamp',inplace =True,kind = 'mergesort',ascending = True)

In [172]:
dataset.drop(columns=['date','ch_code','time_taken','stop','route','price','full_information_timestamp','num_code','flight_day'],axis = 1,inplace=True)

In [173]:
def arranging_dep_time_by_price(time):
    if time =='Early morning':
        return 1
    elif time == 'Afternoon':
        return 2
    elif time == 'Morning':
        return 3
    else :
        return 4

In [174]:
dataset['dep_time'] = dataset.dep_time.apply(lambda time: arranging_dep_time_by_price(time))

In [175]:
dataset['arr_time'] = dataset.arr_time.apply(lambda time: arranging_dep_time_by_price(time))

In [176]:
def arranging_day_by_price(day):
    if day =='Tuesday' :
        return 1
    elif day =='Thursday' :
        return 2
    elif day =='Wednesday' :
        return 3
    elif day =='Friday' :
        return 4
    elif day =='Monday' :
        return 5
    elif day =='Saturday' :
        return 6
    else :
        return 7

In [177]:
dataset['week_day_of_flight'] = dataset.week_day_of_flight.apply(lambda day: arranging_day_by_price(day))

In [178]:
def arranging_airline_by_price(airline):
    if airline == 'Other_airline' :
        return 1
    elif airline == 'Indigo' :
        return 2
    elif airline == 'Air India' :
        return 3
    else :
        return 4

In [179]:
dataset['airline'] = dataset.airline.apply(lambda airline: arranging_airline_by_price(airline))

In [180]:
def arranging_source_by_price(source):
    if source == 'Delhi' :
        return 1
    elif source == 'Hyderabad' :
        return 2
    elif source == 'Mumbai' :
        return 3
    elif source == 'Bangalore' :
        return 4
    elif source == 'Kolkata' :
        return 5
    else :
        return 6

In [181]:
dataset['source'] = dataset.source.apply(lambda source: arranging_source_by_price(source))

In [182]:
def arranging_destination_by_price(source):
    if source == 'Delhi' :
        return 1
    elif source == 'Hyderabad' :
        return 2
    elif source == 'Mumbai' :
        return 3
    elif source == 'Bangalore' :
        return 4
    elif source == 'Chennai' :
        return 5
    else :
        return 6

In [183]:
dataset['destination'] = dataset.destination.apply(lambda destination: arranging_destination_by_price(destination))

In [184]:
dataset

,airline,dep_time,arr_time,type,flight_month,week_day_of_flight,num_of_hours_taken,num_of_stops,distance_between_2_cities,source,destination,formatted_price
6760,1,1,3,0,2,4,2,0,0.653497,1,3,5956
167590,1,1,2,0,2,4,10,1,0.985014,1,4,7423
5914,1,1,2,0,2,4,11,1,0.985014,1,4,21343
94844,1,1,2,0,2,4,7,1,0.985014,1,4,7423
96663,1,1,4,0,2,4,14,1,0.985014,1,4,21343
...,...,...,...,...,...,...,...,...,...,...,...,...
132751,1,4,3,0,3,2,8,1,0.984105,3,6,5817
133933,2,4,3,0,3,2,8,1,0.676203,5,2,3401
154207,1,4,1,0,3,2,3,0,0.985014,4,1,4500
154583,1,4,1,0,3,2,2,0,0.653497,3,1,2339


In [186]:
label = dataset['formatted_price']
dataset.drop(columns=['formatted_price'],axis=1,inplace=True)
features = dataset.copy ()